# Real Embedding Training

In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

In [ ]:
from qiskit import IBMQ

# KQC Token
#TOKEN = "b5b31be2bb5237d52e8bdef143913cd62425eadcbda5c56c6f8ed8d067573ac0e07aa41d23d0bfcc880882a4142e2f812dad876fb3208d5c5f8f321be6626cce"

# DKP Token
TOKEN = "3ae7ac10f40eb88c7ebb0eca20aa0788e7a96da729e2c3848d9864684362aaf50dfdda7cb3ec1ffd75eb7ed5b44f7c14f2f17419cf600ce14437ee7cd00ac75b"


IBMQ.save_account(TOKEN, overwrite=True)
IBMQ.load_account() # Load account from disk


#KQC Provider
#provider = IBMQ.get_provider(hub='ibm-q-kqc', group='internal', project='reservations')
#provider = IBMQ.get_provider(hub='ibm-q-kqc', group='kqc-research', project='kernel-experimen')


#SKKU Provider
provider = IBMQ.get_provider(hub='ibm-q-skku', group='internal', project='reservations')


backend = "ibmq_toronto"

dev = qml.device(
    'qiskit.ibmq',
    wires=4,
    shots=1024,
    backend=backend,
    provider=provider
)

In [ ]:
#steps = 50
batch_size = 10

In [ ]:

def new_data(batch_size, X, Y):
    X1_new, X2_new, Y_new = [], [], []
    for i in range(batch_size):
        n, m = np.random.randint(len(X)), np.random.randint(len(X))
        X1_new.append(X[n])
        X2_new.append(X[m])
        if Y[n] == Y[m]:
            Y_new.append(1)
        else:
            Y_new.append(0)

    X1_new, X2_new, Y_new = torch.tensor(X1_new).to(torch.float32), torch.tensor(X2_new).to(torch.float32), torch.tensor(Y_new).to(torch.float32)
    if feature_reduction == False:
        X1_new = X1_new.permute(0, 3, 1, 2)
        X2_new = X2_new.permute(0, 3, 1, 2)
    return X1_new, X2_new, Y_new

In [ ]:
@qml.qnode(dev, interface="torch", diff_method="parameter-shift")
def embedding_overlap(inputs):

    for i in range(4):
        qml.Hadamard(wires=i)

    embedding.exp_Z(inputs[0], wires=0)
    embedding.exp_Z(inputs[1], wires=1)
    embedding.exp_Z(inputs[2], wires=2)
    embedding.exp_Z(inputs[3], wires=3)
    
    
    embedding.exp_ZZ1(inputs[4], wires=[0,1])
    embedding.exp_ZZ1(inputs[5], wires=[1,2])
    embedding.exp_ZZ1(inputs[6] - inputs[13], wires=[2,3])
    embedding.exp_ZZ1(-inputs[12], wires=[1,2])
    embedding.exp_ZZ1(-inputs[11], wires=[0,1])

    embedding.exp_Z(-1 * inputs[10], wires=3)
    embedding.exp_Z(-1 * inputs[9], wires=2)
    embedding.exp_Z(-1 * inputs[8], wires=1)
    embedding.exp_Z(-1 * inputs[7], wires=0)

    for i in range(4):
        qml.Hadamard(wires=i)

    return qml.probs(wires=range(4))
    

class Model2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer2 = qml.qnn.TorchLayer(embedding_overlap, weight_shapes={})
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
    def forward(self, x1, x2):
        x1 = self.linear_relu_stack2(x1)
        x2 = self.linear_relu_stack2(x2)
        x = torch.concat([x1, x2], 1)
        x = self.qlayer2(x)
        return x[:,0]


class Model3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer3 = qml.qnn.TorchLayer(embedding_overlap, weight_shapes={})
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x1, x2):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x2 = self.layer1(x2)
        x2 = self.layer2(x2)
        x2 = x2.view(-1, 7 * 7)
        x2 = self.fc(x2)

        x = torch.concat([x1, x2], 1)
        x = self.qlayer3(x)
        return x[:,0]

In [ ]:
def train_models(model_name):
    train_loss = []
    if model_name == 'Model2':
        model = Model2()
        #model.load_state_dict(torch.load('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model2 pre-train/Model2_31.pt'))
        #steps = 18

    elif model_name == 'Model3':
        model = Model3()
        model.load_state_dict(torch.load('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model3 pre-train/Model3_26.pt'))
        steps = 23

    model.train()
    loss_fn = torch.nn.MSELoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.1)
    for it in range(steps):

        if model_name == 'Model2':
            PATH_it = f'/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model2 pre-train/Model2_{it+32}.pt'
        elif model_name == 'Model3':
            PATH_it = f'/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model3 pre-train/Model3_{it+27}.pt'

        X1_batch, X2_batch, Y_batch = new_data(batch_size, X_train, Y_train)
        X1_batch, X2_batch, Y_batch = X1_batch, X2_batch, Y_batch

        pred = model(X1_batch, X2_batch)
        pred, Y_batch = pred.to(torch.float32), Y_batch.to(torch.float32)
        loss = loss_fn(pred, Y_batch)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()
        opt.step()
        print(f"Iterations: {it} Loss: {loss.item()}")
        torch.save(model.state_dict(), PATH_it)


# Train

Model 2

In [ ]:
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

train_models('Model2')

Model 3

In [ ]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

train_models('Model3')